In [73]:
# # suggestuions for performance enhanceent:
# use their data generator
# fine tune threshold or alpha margin on evaluation
# further fine tune the remaining parameter on the test data
# then tets on your own snetences,
# maybe add more layers of differenr types,
# maybe improve the test function by using distances squared (i am not sure if it'll work, but late's see)

# Assignment 4:  Question duplicates

Welcome to the fourth assignment of course 3. In this assignment you will explore Siamese networks applied to natural language processing. You will further explore the fundamentals of pytorch and you will be able to implement a more complicated structure using it. By completing this assignment, you will learn how to implement models with different architectures. 


## Table of Contents

- [Overview](#0)
- [1 - Importing the Data](#1)
    - [1.1 - Loading in the Data](#1-1)
    - [1.2 - Converting a Question to a Tensor](#1-2)
    - [1.3 - Understanding the Iterator](#1-3)
        - [Exercise 1 - data_generator (UNQ_C1)](#ex-1)
- [2 - Defining the Siamese Model](#2)
    - [2.1 - Understanding Siamese Network](#2-1)
        - [Exercise 2 - Siamese (UNQ_C2)](#ex-2)
    - [2.2 - Hard Negative Mining](#2-2)
        - [Exercise 3 - TripletLossFn (UNQ_C3)](#ex-3)
- [3 - Training](#3)
    - [3.1 - Training the Model](#3-1)
        - [Exercise 4 - train_model (UNQ_C4)](#ex-4)
- [4 - Evaluation](#4)
    - [4.1 - Evaluating your Siamese Network](#4-1)
    - [4.2 - Classify](#4-2)
        - [Exercise 5 - classify (UNQ_C5)](#ex-5)
- [5 -Testing with your Own Questions](#5)
    - [Exercise 6 - predict (UNQ_C6)](#ex-6)
- [On Siamese Networks](#6)

<a name='0'></a>
## Overview
In this assignment, concretely you will: 

- Learn about Siamese networks
- Understand how the triplet loss works
- Understand how to evaluate accuracy
- Use cosine similarity between the model's outputted vectors
- Use the data generator to get batches of questions
- Predict using your own model

By now, you are familiar with pytorch and know how to make use of classes to define your model. We will start this homework by asking you to preprocess the data the same way you did in the previous assignments. After processing the data you will build a classifier that will allow you to identify whether two questions are the same or not. 
<img src = "images/meme.png" style="width:550px;height:300px;"/>


You will process the data first and then pad in a similar way you have done in the previous assignment. Your model will take in the two question embeddings, run them through an LSTM, and then compare the outputs of the two sub networks using cosine similarity. Before taking a deep dive into the model, start by importing the data set.

<a name='1'></a>
## 1 - Importing the Data

<a name='1-1'></a>
### 1.1 - Loading in the Data

You will be using the Quora question answer dataset to build a model that could identify similar questions. This is a useful task because you don't want to have several versions of the same question posted. Several times when teaching I end up responding to similar questions on piazza, or on other community forums. This data set has been labeled for you. Run the cell below to import some of the packages you will be using. 

In [1]:
import os
import nltk

import numpy as np
import pandas as pd
import random as rnd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# set nltk path
nltk.data.path.append('/nltk_data')

# set random seeds
rnd.seed(34)


You will now load in the data set. We have done some preprocessing for you. If you have taken the deeplearning specialization, this is a slightly different training method than the one you have seen there. If you have not, then don't worry about it, we will explain everything. 

In [2]:
data = pd.read_csv("data/questions.csv")
N    = len(data)

print('Number of question pairs: ', N)

data.head()


Number of question pairs:  404351


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


We first split the data into a train and test set. The test set will be used later to evaluate our model.

In [3]:
N_train = 300000
N_test  = 10*1024

data_train = data[:N_train]
data_test  = data[N_train:N_train+N_test]

print("Train set:", len(data_train), "Test set:", len(data_test))
del(data) # remove to free memory ##############################################################################################


Train set: 300000 Test set: 10240


As explained in the lectures, we select only the question pairs that are duplicate to train the model. <br>
We build two batches as input for the Siamese network and we assume that question $q1_i$ (question $i$ in the first batch) is a duplicate of $q2_i$ (question $i$ in the second batch), but all other questions in the second batch are not duplicates of $q1_i$.  
The test set uses the original pairs of questions and the status describing if the questions are duplicates.

In [4]:
td_index = (data_train['is_duplicate'] == 1).to_numpy()         #####################
td_index = [i for i, x in enumerate(td_index) if x]   # if x is True then return the corresponding i-enumerate-integer-id

print('number of duplicate questions: ', len(td_index))
print('indexes of first ten duplicate questions:', td_index[:10])


number of duplicate questions:  111486
indexes of first ten duplicate questions: [5, 7, 11, 12, 13, 15, 16, 18, 20, 29]


In [518]:
#############################################################################

In [519]:
data_train['is_duplicate'][:7] == 1

0    False
1    False
2    False
3    False
4    False
5     True
6    False
Name: is_duplicate, dtype: bool

In [520]:
(data_train['is_duplicate'][:7] == 1).to_numpy()


array([False, False, False, False, False,  True, False])

In [521]:
type((data_train['is_duplicate'][:7] == 1).to_numpy())


numpy.ndarray

In [522]:
###############################################################################

In [13]:
print(data_train['question1'][5])  #  Example of question duplicates (first one in data)
print(data_train['question2'][5])
print('is_duplicate: ', data_train['is_duplicate'][5])


Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?
is_duplicate:  1


In [14]:
Q1_train_words = np.array(data_train['question1'][td_index])
Q2_train_words = np.array(data_train['question2'][td_index])

Q1_test_words = np.array(data_test['question1'])
Q2_test_words = np.array(data_test['question2'])
y_test        = np.array(data_test['is_duplicate'])


Above, you have seen that you only took the duplicated questions for training our model. <br>You did so on purpose, because the data generator will produce batches $([q1_1, q1_2, q1_3, ...]$, $[q2_1, q2_2,q2_3, ...])$  where $q1_i$ and $q2_k$ are duplicate if and only if $i = k$.

<br>Let's print to see what your data looks like.

In [15]:
print('TRAINING QUESTIONS:\n')
print('Question 1: ', Q1_train_words[0])
print('Question 2: ', Q2_train_words[0], '\n')
print('Question 1: ', Q1_train_words[5])
print('Question 2: ', Q2_train_words[5], '\n')

print('TESTING QUESTIONS:\n')
print('Question 1: ', Q1_test_words[0])
print('Question 2: ', Q2_test_words[0], '\n')
print('is_duplicate =', y_test[0], '\n')


TRAINING QUESTIONS:

Question 1:  Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
Question 2:  I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me? 

Question 1:  What would a Trump presidency mean for current international master’s students on an F1 visa?
Question 2:  How will a Trump presidency affect the students presently in US or planning to study in US? 

TESTING QUESTIONS:

Question 1:  How do I prepare for interviews for cse?
Question 2:  What is the best way to prepare for cse? 

is_duplicate = 0 



You will now encode each word of the selected duplicate pairs with an index. <br> Given a question, you can then just encode it as a list of numbers.  

First you tokenize the questions using `nltk.word_tokenize`. <br>
You need a python default dictionary which later, during inference, assigns the values $0$ to all Out Of Vocabulary (OOV) words.<br>
Then you encode each word of the selected duplicate pairs with an index. Given a question, you can then just encode it as a list of numbers. 

In [16]:
##################################################################

In [17]:
Q1_train_words

array(['Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
       'How can I be a good geologist?',
       'How do I read and find my YouTube comments?', ...,
       'What are the top 10 TV series one should genuinely watch?',
       'Is there no life on other planets?',
       'How do I tell the difference between infatuation and love?'],
      dtype=object)

In [18]:
Q1_train_words.shape

(111486,)

In [19]:
type(Q1_train_words)

numpy.ndarray

In [20]:
a = "Hello world!"
b = 'How are you!'
c = [a,b]
d = np.array(c)
d

array(['Hello world!', 'How are you!'], dtype='<U12')

In [21]:
e = np.empty_like(d)
e # like d's structure, e is an empty array, also the dtypes are also same


array(['', ''], dtype='<U12')

In [22]:
# Q1_train.shape

NameError: name 'Q1_train' is not defined

In [533]:
##################################################################

In [24]:
#create arrays
# 4aron sets of sentences k corresponding structures and dtypes k hisaab say 4 empty arrays initialize kr deen 

Q1_train = np.empty_like(Q1_train_words)
Q2_train = np.empty_like(Q2_train_words)

Q1_test = np.empty_like(Q1_test_words)
Q2_test = np.empty_like(Q2_test_words)


In [25]:
# Building the vocabulary with the train set         (this might take a minute)
from collections import defaultdict            # agar is dict ko koi aesi key pass kro gay, jo is k andar mojood nahi hay tu ye 0 return kray gi

vocab = defaultdict(lambda: 0)
vocab['<PAD>'] = 1

for idx in range(len(Q1_train_words)):
    Q1_train[idx] = nltk.word_tokenize(Q1_train_words[idx])          # ye vo jo empty arrays theen on ki fill krna start kia
    Q2_train[idx] = nltk.word_tokenize(Q2_train_words[idx])
    
    q = Q1_train[idx] + Q2_train[idx]
    
    for word in q:
        if word not in vocab:
            vocab[word] = len(vocab) + 1
print('The length of the vocabulary is: ', len(vocab))


The length of the vocabulary is:  36268


In [588]:
############################################################

In [26]:
Q1_train.shape


(111486,)

In [28]:
print(Q1_train[0])


['Astrology', ':', 'I', 'am', 'a', 'Capricorn', 'Sun', 'Cap', 'moon', 'and', 'cap', 'rising', '...', 'what', 'does', 'that', 'say', 'about', 'me', '?']


In [29]:
# Q1_train aik 1d array hi hay, but ....
# so, nltk.word_tokenize(Q1_train_words[idx]), nay aik list return ki, or Q1_train k elements ye lists hain...
# 1d array of lists-of-words


In [30]:
print(Q1_train[0]+Q2_train[0])
# ye lists concatenate ho gaieen
# tu jab, np.arrays k corresponding elements lists hoon, tu vo sum nahi concatenate hotay hain...


['Astrology', ':', 'I', 'am', 'a', 'Capricorn', 'Sun', 'Cap', 'moon', 'and', 'cap', 'rising', '...', 'what', 'does', 'that', 'say', 'about', 'me', '?', 'I', "'m", 'a', 'triple', 'Capricorn', '(', 'Sun', ',', 'Moon', 'and', 'ascendant', 'in', 'Capricorn', ')', 'What', 'does', 'this', 'say', 'about', 'me', '?']


In [31]:
vocab['how r u']
# default dict thi, is liye 0 return kia, jab key present nahi thi dict main


0

In [594]:
############################################################

In [32]:
print(vocab['<PAD>'])
print(vocab['Astrology'])
print(vocab['Astronomy'])  #not in vocabulary, returns 0


1
2
0


In [33]:
for idx in range(len(Q1_test_words)): 
    Q1_test[idx] = nltk.word_tokenize(Q1_test_words[idx])
    Q2_test[idx] = nltk.word_tokenize(Q2_test_words[idx])


In [34]:
print('Train set has reduced to: ', len(Q1_train) ) # pehlay bhi yehi length thi...
print('Test set length: ', len(Q1_test) ) 


Train set has reduced to:  111486
Test set length:  10240


<a name='1-2'></a>
### 1.2 - Converting a Question to a Tensor

You will now convert every question to a tensor, or an array of numbers, using your vocabulary built above.

In [35]:
# Converting questions to array of integers
for i in range(len(Q1_train)):                             # outer loop
    Q1_train[i] = [vocab[word] for word in Q1_train[i]]    # sub inner loop
    Q2_train[i] = [vocab[word] for word in Q2_train[i]]    # serially-parallel-level sub-inner-loop-2
                                                           # & so on ...
        
for i in range(len(Q1_test)):
    Q1_test[i] = [vocab[word] for word in Q1_test[i]]      # ab har element par list of words ki bjaye, list of numbers pri hui hay
    Q2_test[i] = [vocab[word] for word in Q2_test[i]]


In [37]:
print('first question in the train set:\n')
print(Q1_train_words[0], '\n') 
print('encoded version:')
print(Q1_train[0],'\n')

print('first question in the test set:\n')
print(Q1_test_words[0], '\n')
print('encoded version:')
print(Q1_test[0]) 


first question in the train set:

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me? 

encoded version:
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21] 

first question in the test set:

How do I prepare for interviews for cse? 

encoded version:
[32, 38, 4, 107, 65, 1015, 65, 11509, 21]


You will now split your train set into a training/validation set so that you can use it to train and evaluate your Siamese model.

In [38]:
# Splitting the data
cut_off = int(len(Q1_train)*.8)

train_Q1, train_Q2 = Q1_train[:cut_off],   Q2_train[:cut_off]
val_Q1,     val_Q2 = Q1_train[cut_off: ],  Q2_train[cut_off:]

print('Number of duplicate questions: ', len(Q1_train))
print("The length of the training set is:  ", len(train_Q1))
print("The length of the validation set is: ", len(val_Q1))


Number of duplicate questions:  111486
The length of the training set is:   89188
The length of the validation set is:  22298


<a name='1-3'></a>
### 1.3 - Understanding the Iterator 

Most of the time in Natural Language Processing, and AI in general we use batches when training our data sets. If you were to use stochastic gradient descent with one example at a time, it will take you forever to build a model. In this example, we show you how you can build a data generator that takes in $Q1$ and $Q2$ and returns a batch of size `batch_size`  in the following format $([q1_1, q1_2, q1_3, ...]$, $[q2_1, q2_2,q2_3, ...])$. The tuple consists of two arrays and each array has `batch_size` questions. Again, $q1_i$ and $q2_i$ are duplicates, but they are not duplicates with any other elements in the batch. 

<br>

The command ```next(data_generator)```returns the next batch. This iterator returns the data in a format that you could directly use in your model when computing the feed-forward of your algorithm. This iterator returns a pair of arrays of questions. 

<a name='ex-1'></a>
### Exercise 1 - data_generator

**Instructions:**  
Implement the data generator below. Here are some things you will need. 

- While true loop.
- if `index >= len_Q1`, set the `idx` to $0$.
- The generator should return shuffled batches of data. To achieve this without modifying the actual question lists, a list containing the indexes of the questions is created. This list can be shuffled and used to get random batches everytime the index is reset.
- Append elements of $Q1$ and $Q2$ to `input1` and `input2` respectively.
- if `len(input1) == batch_size`, determine `max_len` as the longest question in `input1` and `input2`. Ceil `max_len` to a power of $2$ (for computation purposes) using the following command:  `max_len = 2**int(np.ceil(np.log2(max_len)))`.
- Pad every question by `vocab['<PAD>']` until you get the length `max_len`.
- Use yield to return `input1, input2`. 
- Don't forget to reset `input1, input2`  to empty arrays at the end (data generator resumes from where it last left).

In [601]:
###################################################################################

In [39]:
def data_generator(train_Q1_, train_Q2_, batch_size_, pad=1):
    counter_1 = 0
    counter_2 = batch_size_

    while True:
        batch_of_inputs_1 = []
        batch_of_inputs_2 = []
        
        batch_of_inputs_1 = train_Q1_[counter_1:counter_2].copy()
        batch_of_inputs_2 = train_Q2_[counter_1:counter_2].copy()

        max_len_in_batch_of_inputs_1 = 0
        for current_tensor in batch_of_inputs_1:
            if len(current_tensor) > max_len_in_batch_of_inputs_1:
                max_len_in_batch_of_inputs_1 = len(current_tensor)

        max_len_in_batch_of_inputs_2 = 0
        for current_tensor in batch_of_inputs_2:
            if len(current_tensor) > max_len_in_batch_of_inputs_2:
                max_len_in_batch_of_inputs_2 = len(current_tensor)

        for i, current_tensor in enumerate(batch_of_inputs_1):
            current_tensor = current_tensor + (max_len_in_batch_of_inputs_1 - len(current_tensor))*[1]
            batch_of_inputs_1[i] = np.array(current_tensor)

        for i, current_tensor in enumerate(batch_of_inputs_2):
            current_tensor = current_tensor + (max_len_in_batch_of_inputs_2 - len(current_tensor))*[1]
            batch_of_inputs_2[i] = np.array(current_tensor)

        yield np.vstack(batch_of_inputs_1), np.vstack(batch_of_inputs_2)

        counter_1+=batch_size_
        counter_2+=batch_size_
        if counter_2 >= len(train_Q1_):
                counter_1 = 0
                counter_2 = batch_size_
        batch_of_inputs_1 = []
        batch_of_inputs_2 = []



In [40]:
5*[1] + [1,2,3]

[1, 1, 1, 1, 1, 1, 2, 3]

In [41]:
train_generator = data_generator(train_Q1, train_Q2, 2, 1)


In [42]:
a, b = next(train_generator)
a, b


(array([[ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21],
        [32, 33,  4, 34,  6, 35, 36, 21,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1]]),
 array([[ 4, 22,  6, 23,  7, 24,  8, 25, 26, 11, 27, 28,  7, 29, 30, 16,
         31, 18, 19, 20, 21],
        [30, 37,  4, 38, 39, 34,  6, 40, 36, 21,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1]]))

In [43]:
a.shape, b.shape


((2, 20), (2, 21))

In [44]:
a, b = next(train_generator)
a, b


(array([[32, 38,  4, 41, 11, 42, 43, 44, 45, 21],
        [30, 33, 49, 50, 51, 39, 52, 21,  1,  1]]),
 array([[32, 33,  4, 46, 47, 43, 48, 45, 21],
        [32, 33, 53, 49, 54, 51, 39, 52, 21]]))

In [45]:
a.shape, b.shape


((2, 10), (2, 9))

In [633]:
train_Q1[3]


[30, 33, 49, 50, 51, 39, 52, 21]

In [ ]:
###################################################################################

In [467]:
# # UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# # GRADED FUNCTION: data_generator
# def data_generator(Q1, Q2, batch_size, pad=1, shuffle=True):
#     """Generator function that yields batches of data

#     Args:
#         Q1 (list): List of transformed (to tensor) questions.
#         Q2 (list): List of transformed (to tensor) questions.
#         batch_size (int): Number of elements per batch.
#         pad (int, optional): Pad character from the vocab. Defaults to 1.
#         shuffle (bool, optional): If the batches should be randomnized or not. Defaults to True.
#     Yields:
#         tuple: Of the form (input1, input2) with types (numpy.ndarray, numpy.ndarray)
#         NOTE: input1: inputs to your model [q1a, q2a, q3a, ...] i.e. (q1a,q1b) are duplicates
#               input2: targets to your model [q1b, q2b,q3b, ...] i.e. (q1a,q2i) i!=a are not duplicates
#     """

#     input1 = []
#     input2 = []
#     idx = 0
#     len_q = len(Q1)
#     question_indexes = [*range(len_q)]
    
#     if shuffle:
#         rnd.shuffle(question_indexes)
    
#     ### START CODE HERE (Replace instances of 'None' with your code) ###
#     while True:
#         if idx >= len_q:
#             # if idx is greater than or equal to len_q, set idx accordingly 
#             # (Hint: look at the instructions above)
#             idx = 0
#             # shuffle to get random batches if shuffle is set to True
#             if shuffle:
#                 rnd.shuffle(question_indexes) 
        
#         # get questions at the `question_indexes[idx]` position in Q1 and Q2
#         q1 = Q1[question_indexes[idx]]
#         q2 = Q2[question_indexes[idx]]
        
#         # increment idx by 1
#         idx += 1
#         # append q1
#         input1.append(q1)
#         # append q2
#         input2.append(q2)
#         if len(input1) == batch_size:
#             # determine max_len as the longest question in input1 & input 2
#             # Hint: use the `max` function. 
#             # take max of input1 & input2 and then max out of the two of them.
#             max_len = max(max([len(q) for q in input1]), max([len(q) for q in input2]))
#             # pad to power-of-2 (Hint: look at the instructions above)
#             max_len = 2**int(np.ceil(np.log2(max_len)))
#             b1 = [] 
#             b2 = [] 
#             for q1, q2 in zip(input1, input2):
#                 # add [pad] to q1 until it reaches max_len
#                 q1 = q1 + [pad] * (max_len - len(q1))
#                 # add [pad] to q2 until it reaches max_len
#                 q2 = q2 + [pad] * (max_len - len(q2))                
#                 # append q1
#                 b1.append(q1)
#                 # append q2
#                 b2.append(q2)
#             # use b1 and b2
#             yield np.array(b1), np.array(b2)
#     ### END CODE HERE ###
#             # reset the batches
#             input1, input2 = [], []  # reset the batches


In [468]:
# batch_size = 2
# res1, res2 = next(data_generator(train_Q1, train_Q2, batch_size))
# print("First questions  : ",'\n', res1, '\n')
# print("Second questions : ",'\n', res2)


First questions  :  
 [[   30    87   669    11   670   131   525    11   418   780   527  1507
     21     1     1     1     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1]
 [   30    87   116  3141  6925    65    78  1633  2834   131 24752    65
    140   421  1673    71    21     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1]] 

Second questions :  
 [[  32   76   78  741   72 4324  525   11  418  534 1261   77  308   21
    30   87   78  669   11  670   21    1    1    1    1    1    1    1
     1    1    1    1]
 [  30   87  216  567 6925   65 6488 1633 3229   28  140  421   21    1
     1    1    1    1    1    1    1    1    1    1    1    1    1    1
     1    1    1    1]]


**Note**: The following expected output is valid only if you run the above test cell **_once_** (first time). The output will change on each execution.

If you think your implementation is correct and it is not matching the output, make sure to restart the kernel and run all the cells from the top again. 

**Expected Output:**
```CPP
First questions  :  
 [[  30   87   78  134 2132 1981   28   78  594   21    1    1    1    1
     1    1]
 [  30   55   78 3541 1460   28   56  253   21    1    1    1    1    1
     1    1]] 

Second questions :  
 [[  30  156   78  134 2132 9508   21    1    1    1    1    1    1    1
     1    1]
 [  30  156   78 3541 1460  131   56  253   21    1    1    1    1    1
     1    1]]
```
Now that you have your generator, you can just call it and it will return tensors which correspond to your questions in the Quora data set.<br>Now you can go ahead and start building your neural network. 



## Defining the Siamese Model


### - Understanding Siamese Network 
A Siamese network is a neural network which uses the same weights while working in tandem on two different input vectors to compute comparable output vectors.The Siamese network you are about to implement looks like this:

<img src = "images/siamese.png" style="width:600px;height:300px;"/>

You get the question embedding, run it through an LSTM layer, normalize $v_1$ and $v_2$, and finally use a triplet loss (explained below) to get the corresponding cosine similarity for each pair of questions. As usual, you will start by importing the data set. The triplet loss makes use of a baseline (anchor) input that is compared to a positive (truthy) input and a negative (falsy) input. The distance from the baseline (anchor) input to the positive (truthy) input is minimized, and the distance from the baseline (anchor) input to the negative (falsy) input is maximized. In math equations, you are trying to maximize the following.

$$\mathcal{L}(A, P, N)=\max \left(\|\mathrm{f}(A)-\mathrm{f}(P)\|^{2}-\|\mathrm{f}(A)-\mathrm{f}(N)\|^{2}+\alpha, 0\right)$$

$A$ is the anchor input, for example $q1_1$, $P$ the duplicate input, for example, $q2_1$, and $N$ the negative input (the non duplicate question), for example $q2_2$.<br>
$\alpha$ is a margin; you can think about it as a safety net, or by how much you want to push the duplicates from the non duplicates. 
<br>


### Exercise - Siamese


In [261]:
#########################################################

In [46]:
len(vocab)


41699

In [263]:
#########################################################

In [47]:
vocab_size     = len(vocab)
embedding_out  = 128
n_layers       = 3

class LSTM_Net(nn.Module):
    def __init__(self, vocab_size_, embedding_out_, n_layers_):
        super(LSTM_Net, self).__init__()
        
        self.embedding = nn.Embedding(num_embeddings=vocab_size_, embedding_dim=embedding_out_)
        self.lstm      = nn.LSTM(input_size=embedding_out_, hidden_size=embedding_out_, num_layers=n_layers_, batch_first=True)

    
    def forward(self, x, hidden, memmory_cell):
        embedded_output                     = self.embedding(x)
        lstm_output, (hidden, memmory_cell) = self.lstm(embedded_output, (hidden, memmory_cell))
        
        meaned_output                       = lstm_output.mean(dim=1)                  # meaned features for each sequence. mean of corresponding-embedding-features of all words of a sequence 
        l2_normalized_output                = F.normalize(meaned_output, dim=-1, p=2)  # L2 normalized rows | the L2 norm of each row will be 1
        
        return l2_normalized_output, hidden, memmory_cell

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LSTM_Net(vocab_size, embedding_out, n_layers).to(device)
print(model)


LSTM_Net(
  (embedding): Embedding(41699, 128)
  (lstm): LSTM(128, 128, num_layers=3, batch_first=True)
)


# Checking the inputs and ouputs of the model


In [265]:
batch_size = 256
vocab_size=len(vocab)
torch.manual_seed(0)
input_sequences = torch.randint(0, vocab_size, (batch_size, 10))
print('input_sequences: ', input_sequences.shape)


input_sequences:  torch.Size([256, 10])


In [266]:
vocab_size     = len(vocab)
embedding_out = 128

# embedding layer
embedding_ = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_out)
embed_out_ = embedding_(input_sequences)

print('embed_out: ', embed_out_.shape)


embed_out:  torch.Size([256, 10, 128])


In [267]:
# GRU layer

# Defining the LSTM parameters
input_size  = embedding_out  # Number of features in the input
hidden_size = embedding_out # Number of features in the hidden state
n_layers  = 1   # Number of recurrent layers


# Create an LSTM layer
lstm_ = nn.LSTM(input_size, hidden_size, n_layers, batch_first=True)


# Optionally, create initial hidden and cell states
h0 = torch.zeros(n_layers, batch_size, hidden_size)
c0 = torch.zeros(n_layers, batch_size, hidden_size)


# Forward pass through the LSTM
lstm_out_, (hn, cn) = lstm_(embed_out_, (h0, c0))

# Print the shapes of the outputs
print("Output shape:", lstm_out_.shape)  # Output shape: (batch_size, seq_len, num_directions * hidden_size)
print("hn shape:", hn.shape)          # hn shape: (num_layers * num_directions, batch_size, hidden_size)
print("cn shape:", cn.shape)          # cn shape: (num_layers * num_directions, batch_size, hidden_size)


Output shape: torch.Size([256, 10, 128])
hn shape: torch.Size([1, 256, 128])
cn shape: torch.Size([1, 256, 128])


In [268]:
mean_out = lstm_out_.mean(dim=1)
mean_out.shape


torch.Size([256, 128])

In [269]:
l2_normalized_out = F.normalize(mean_out, dim=-1, p=2)
l2_normalized_out.shape


torch.Size([256, 128])

In [270]:
l2_normalized_out[0,:10]


tensor([-0.0535,  0.1342,  0.0963, -0.0487, -0.0115, -0.1075,  0.0891,  0.1616,
         0.1008,  0.0173], grad_fn=<SliceBackward0>)

In [271]:
l2_norm = torch.norm(l2_normalized_out[0,:], p=2)
l2_norm

# perfect!


tensor(1., grad_fn=<LinalgVectorNormBackward0>)

#### Comparing the final-layer-by-layer-output with the model's output

In [272]:


# from model
l2_normalized_out_model, _ ,_ = model(input_sequences, h0, c0)
l2_normalized_out_model[0,:10]


tensor([ 0.0246, -0.0611,  0.1375, -0.0348,  0.0479, -0.0559, -0.0248,  0.1212,
         0.0304,  0.0172], grad_fn=<SliceBackward0>)

In [273]:
l2_normalized_out_model.shape

torch.Size([256, 128])

In [274]:
l2_norm_model = torch.norm(l2_normalized_out_model[0,:], p=2)
l2_norm_model

# perfect!

tensor(1.0000, grad_fn=<LinalgVectorNormBackward0>)

<a name='2-2'></a>
### 2.2 - Hard Negative Mining


You will now implement the `TripletLoss`.<br>
As explained in the lecture, loss is composed of two terms. One term utilizes the mean of all the non duplicates, the second utilizes the *closest negative*. Our loss expression is then:
 
\begin{align}
 \mathcal{Loss_{1}(A,P,N)} &=\max \left( -cos(A,P)  + mean_{neg} +\alpha, 0\right) \\
 \mathcal{Loss_{2}(A,P,N)} &=\max \left( -cos(A,P)  + closest_{neg} +\alpha, 0\right) \\
\mathcal{Loss(A,P,N)} &= mean(Loss_1 + Loss_2) \\
\end{align}


Further, two sets of instructions are provided. The first set provides a brief description of the task. If that set proves insufficient, a more detailed set can be displayed.  

### Exercise - TripletLossFn


In [696]:
# Their input
# v1 = np.array([[ 0.26726124,  0.53452248,  0.80178373],[-0.5178918 , -0.57543534, -0.63297887]])
# v2 = np.array([[0.26726124, 0.53452248, 0.80178373],[0.5178918 , 0.57543534, 0.63297887]])
# print("Triplet Loss:", TripletLossFn(v1,v2))


#### Their output
Triplet Loss: 0.7035077

**Expected Output:**
```CPP
Triplet Loss: ~ 0.70
```   

In [ ]:
##################################################################################################

##  loss fn in numpy - step by step, see c3w4 nb2 loss fn in numpy explanation


In [275]:
v1 = np.array([[ 0.26726124,  0.53452248,  0.80178373],[-0.5178918 , -0.57543534, -0.63297887]])  # ye already normalized hain
v2 = np.array([[0.26726124, 0.53452248, 0.80178373],[0.5178918 , 0.57543534, 0.63297887]])

sim = np.dot(v1, v2.T)
sim

array([[ 1.        ,  0.95350769],
       [-0.95350769, -1.        ]])

In [276]:
# Batch size
b = sim.shape[0]
b

2

In [277]:
sim_ap = np.diag(sim)
sim_ap


array([ 1., -1.])

In [278]:
sim_an = sim - np.diag(sim_ap)
sim_an


array([[ 0.        ,  0.95350769],
       [-0.95350769,  0.        ]])

In [279]:
mean_neg = np.sum(sim_an, axis=-1, keepdims=True) / (b - 1)
mean_neg


array([[ 0.95350769],
       [-0.95350769]])

In [280]:
mask_1 = np.identity(b) == 1            # mask to exclude the diagonal
mask_1


array([[ True, False],
       [False,  True]])

In [281]:
mask_2 = sim_an > sim_ap.reshape(b, 1)  # mask to exclude sim_an > sim_ap
mask_2


array([[False, False],
       [ True,  True]])

In [282]:
mask = mask_1 | mask_2
mask


array([[ True, False],
       [ True,  True]])

In [283]:
sim_an_masked = np.copy(sim_an)         # create a copy to preserve sim_an
sim_an_masked[mask] = -2
sim_an_masked


array([[-2.        ,  0.95350769],
       [-2.        , -2.        ]])

In [284]:
closest_neg = np.max(sim_an_masked, axis=1, keepdims=True)
closest_neg


array([[ 0.95350769],
       [-2.        ]])

In [285]:
# Alpha margin
alpha = 0.25

# Modified triplet loss
# Loss 1
l_1 = np.maximum(mean_neg - sim_ap.reshape(b, 1) + alpha, 0)
l_1


array([[0.20350769],
       [0.29649231]])

In [286]:
# Loss 2
l_2 = np.maximum(closest_neg - sim_ap.reshape(b, 1) + alpha, 0)
l_2


array([[0.20350769],
       [0.        ]])

In [287]:
# Loss full
l_full = l_1 + l_2
l_full


array([[0.40701537],
       [0.29649231]])

In [288]:
# Cost
cost = np.sum(l_full)
cost
#perfect


0.7035076825158911

## loss fn in torch

In [49]:
v1 = torch.tensor([[ 0.26726124,  0.53452248,  0.80178373],[-0.5178918 , -0.57543534, -0.63297887]])  # ye already normalized hain
v2 = torch.tensor([[0.26726124, 0.53452248, 0.80178373],[0.5178918 , 0.57543534, 0.63297887]])

sim = torch.matmul(v1, v2.T)  # pairwise cosine sim
sim


tensor([[ 1.0000,  0.9535],
        [-0.9535, -1.0000]])

In [50]:
# Batch size
b = sim.shape[0]
b


2

In [51]:
sim_ap = torch.diag(sim)  # the positive ones (duplicates)
sim_ap


tensor([ 1., -1.])

In [52]:
sim_an = sim - torch.diag(sim_ap)
sim_an


tensor([[ 0.0000,  0.9535],
        [-0.9535,  0.0000]])

In [53]:
mean_neg = torch.sum(sim_an, dim=-1, keepdim=True) / (b - 1)   # direct mean na lena, vahan bt default 'b' say divide hoga, hum nay (b-1) say divide krna hay
# also notice: it's, keepdim NOT keepdimS  !!!
 # keepdimS will also work but will later cause problems
mean_neg


tensor([[ 0.9535],
        [-0.9535]])

In [54]:
mask_1 = torch.eye(b, device=sim.device) == 1 # mask to exclude the diagonal           # mask_1 = torch.eye(b) == 1
mask_1


tensor([[ True, False],
        [False,  True]])

In [55]:
mask_2 = sim_an > sim_ap.reshape(b, 1)  # mask to exclude sim_an > sim_ap
mask_2


tensor([[False, False],
        [ True,  True]])

In [56]:
mask = mask_1 | mask_2
mask


tensor([[ True, False],
        [ True,  True]])

In [60]:
# replacing the Trues from mask with -2
sim_an_masked = sim_an.clone()  # create a copy to preserve sim_an
sim_an_masked[mask] = -2


# sim_an_masked = torch.clone(sim_an)         # create a copy to preserve sim_an

sim_an_masked


tensor([[-2.0000,  0.9535],
        [-2.0000, -2.0000]])

In [61]:
closest_neg = torch.max(sim_an_masked, axis=1, keepdim=True).values                ############################# closest_neg = torch.max(sim_an_masked, axis=1, keepdims=True)
                                                                                   ####################closest_neg = closest_neg.values                                             ###########
closest_neg


tensor([[ 0.9535],
        [-2.0000]])

In [64]:
# Alpha margin
alpha = 0.25

# Modified triplet loss
# Loss 1
# l_1 = torch.maximum(mean_neg - sim_ap.reshape(b, 1) + alpha, torch.tensor(0)) ##############
l_1 = torch.maximum(mean_neg - sim_ap.reshape(b, 1) + alpha, torch.tensor(0.0, device=sim.device))

l_1


tensor([[0.2035],
        [0.2965]])

In [66]:
# Loss 2
#l_2 = torch.maximum(closest_neg - sim_ap.reshape(b, 1) + alpha, torch.tensor(0)) ##################
l_2 = torch.maximum(closest_neg - sim_ap.reshape(b, 1) + alpha, torch.tensor(0.0, device=sim.device))
l_2


tensor([[0.2035],
        [0.0000]])

In [67]:
# Loss full
l_full = l_1 + l_2
l_full


tensor([[0.4070],
        [0.2965]])

In [68]:
# Cost
cost = torch.sum(l_full)
cost
#perfect

tensor(0.7035)

## Complete TripletLossFn (pytorch)


In [69]:
# import torch

def TripletLossFn(v1_, v2_, margin_=0.25):
    """Custom Loss function.

    Args:
        v1 (torch.Tensor): Tensor with dimension (batch_size, model_dimension) associated to Q1.
        v2 (torch.Tensor): Tensor with dimension (batch_size, model_dimension) associated to Q2.
        margin (float, optional): Desired margin. Defaults to 0.25.

    Returns:
        torch.Tensor: Triplet Loss.
    """
    # use torch to take the dot product of the two batches (don't forget to transpose the second argument)
    sim = torch.matmul(v1_, v2_.T)  # pairwise cosine sim
    # calculate new batch size
    b   = sim.shape[0]
    # use torch to grab all positive `diagonal` entries in `scores`
    sim_ap = torch.diag(sim)  # the positive ones (duplicates)
    # all similarities between the anchor(positive) and the negative sentences
    sim_an = sim - torch.diag(sim_ap)


    # taking the mean of each row of similarities matrix, excluding diag entries
    mean_neg = torch.sum(sim_an, dim=-1, keepdims=True) / (b - 1)  # direct mean na lena, vahan bt default 'b' say divide hoga, hum nay (b-1) say divide krna hay

    
    # mask to exclude the diagonal from similarities matrix
    mask_1 = torch.eye(b, device=sim.device) == 1
    # mask to exclude sim_an > sim_ap | i.e. all the similaritiesin each row that are greater than the similarity b/w duplicates for that row
    mask_2 = sim_an > sim_ap.reshape(b, 1)


    # combining the two masks            | False*False=False else, always True, e.g: False*True=True .....
    mask          = mask_1 | mask_2

    
    # replacing the Trues from mask with -2
    sim_an_masked = torch.clone(sim_an)         # create a copy to preserve sim_an
    sim_an_masked[mask] = -2


    # return the largest value in a row
    closest_neg = torch.max(sim_an_masked, axis=1, keepdims=True)
    closest_neg = closest_neg.values                                             ########### notice -----.values
    

    # Loss 1
    l_1 = torch.maximum(mean_neg - sim_ap.reshape(b, 1) + alpha, torch.tensor(0)) ######### notice torch.tensor()
    # Loss 2
    l_2 = torch.maximum(closest_neg - sim_ap.reshape(b, 1) + alpha, torch.tensor(0)) ######
    # Loss full
    l_full = l_1 + l_2
    # Cost
    triplet_loss = torch.sum(l_full)
    
    return triplet_loss


# Example usage
v1 = torch.tensor([[0.26726124, 0.53452248, 0.80178373], [-0.5178918, -0.57543534, -0.63297887]])
v2 = torch.tensor([[0.26726124, 0.53452248, 0.80178373], [0.5178918, 0.57543534, 0.63297887]])
print("Triplet Loss:", TripletLossFn(v1, v2))


Triplet Loss: tensor(0.7035)


### Making a TripletLoss layer


In [70]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F


# Define the TripletLoss module
class TripletLoss(nn.Module):
    def __init__(self):
        super(TripletLoss, self).__init__()

    def forward(self, v1_, v2_, LossFn, margin_=0.25):
        return LossFn(v1_, v2_, margin_)


# # Example usage
# batch_of_output_vectors_1 = torch.randn(10, 128, requires_grad=True)  # Example tensor with requires_grad=True
# batch_of_output_vectors_2 = torch.randn(10, 128, requires_grad=True)  # Example tensor with requires_grad=True

# criterion = TripletLoss()
# optimizer = torch.optim.Adam([batch_of_output_vectors_1, batch_of_output_vectors_2], lr=0.001)

# loss = criterion(batch_of_output_vectors_1, batch_of_output_vectors_2, TripletLossFn, 0.25)
# loss.backward()
# optimizer.step()
# loss.item()


criterion = TripletLoss()
loss = criterion(v1, v2, TripletLossFn, 0.25)
loss.item()
# perfect


0.703507661819458

In [ ]:
##################################################################################################

# Training

Now you are going to train your model. As usual, you have to define the cost function and the optimizer. You also have to feed in the built model. Before, going into the training, we will use a special data set up. We will define the inputs using the data generator we built above. The lambda function acts as a seed to remember the last batch that was given. Run the cell below to get the question pairs inputs. 

## Testing Data Generators

In [71]:
# batch_size = 256
# train_generator = data_generator(train_Q1, train_Q2, batch_size, vocab['<PAD>'], shuffle=False)
# val_generator   = data_generator(val_Q1,   val_Q2,   batch_size, vocab['<PAD>'], shuffle=False)

# print('train_Q1.shape ', train_Q1.shape)
# print('val_Q1.shape   ', val_Q1.shape)


TypeError: data_generator() got an unexpected keyword argument 'shuffle'

In [429]:
# a, b = next(train_generator)
# a, b


(array([[   2,    3,    4, ...,    1,    1,    1],
        [  32,   33,    4, ...,    1,    1,    1],
        [  32,   38,    4, ...,    1,    1,    1],
        ...,
        [  30,   87,   78, ...,    1,    1,    1],
        [  32,   16,  224, ...,    1,    1,    1],
        [  30, 1238,  315, ...,    1,    1,    1]]),
 array([[   4,   22,    6, ...,    1,    1,    1],
        [  30,   37,    4, ...,    1,    1,    1],
        [  32,   33,    4, ...,    1,    1,    1],
        ...,
        [  30,   87,  116, ...,    1,    1,    1],
        [  32,   16,  224, ...,    1,    1,    1],
        [  30, 1238,  315, ...,    1,    1,    1]]))

In [430]:
# type(a)


numpy.ndarray

In [431]:
# a.shape, b.shape


((256, 32), (256, 32))

In [433]:
# a, b = next(train_generator)
# a, b

(array([[  30,  156,   78, ...,    1,    1,    1],
        [  30,   55,  224, ...,    1,    1,    1],
        [ 168, 1747,  636, ...,    1,    1,    1],
        ...,
        [  32,   37,    4, ...,    1,    1,    1],
        [ 676,    4,   33, ...,    1,    1,    1],
        [  32,  156, 1051, ...,    1,    1,    1]]),
 array([[ 30, 156,  78, ...,   1,   1,   1],
        [443, 138, 218, ...,   1,   1,   1],
        [244, 636,  78, ...,   1,   1,   1],
        ...,
        [ 30,  87,  78, ...,   1,   1,   1],
        [676,  33,   4, ...,   1,   1,   1],
        [ 30, 156,  78, ...,   1,   1,   1]]))

In [434]:
# a.shape, b.shape


((256, 64), (256, 64))

# Optimizer and Loss Function


In [72]:
# import torch.optim as optim

criterion = TripletLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training Loop

In [790]:
batch_size  = 256
# resetting training gen
train_generator = data_generator(train_Q1, train_Q2, batch_size, vocab['<PAD>'])

n_epochs = 10
steps_per_epoch = len(train_Q1)//batch_size

margin = 0.25

# setting up initial hidden and cell state
num_layers  = 3
batch_size  = batch_size
hidden_size = 128
# Optionally, create initial hidden and cell states
hidden_state = torch.zeros(num_layers, batch_size, hidden_size).to(device)
memmory_cell = torch.zeros(num_layers, batch_size, hidden_size).to(device)
# hidden_state.shape



for epoch in range(n_epochs):
    model.train()                  # agar evaluation accuracy etc bhi check kr rahay hotay tu i guess model ko dobara is tarin mode main dalna zroori tha
    total_loss=0.0

    i=0
    
    for batch_of_inputs_1, batch_of_inputs_2 in train_generator:
        i+=1
        batch_of_inputs_1, batch_of_inputs_2 = torch.from_numpy(batch_of_inputs_1).long().to(device), torch.from_numpy(batch_of_inputs_2).long().to(device)

        optimizer.zero_grad()
        
        # Forward pass
        batch_of_output_vectors_1, _, _ = model(batch_of_inputs_1, hidden_state, memmory_cell)     # OR batch_of_output_vectors_1, hidden_state, memmory_cell
        batch_of_output_vectors_2, _, _ = model(batch_of_inputs_2, hidden_state, memmory_cell)     # OR batch_of_output_vectors_2, hidden_state, memmory_cell

        
        # Computing the loss
        loss = criterion(batch_of_output_vectors_1, batch_of_output_vectors_2, TripletLossFn)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()


        total_loss += loss.item()

        if i % 50 == 0:
            print('i: ',i)
            print('loss: ', total_loss / (i + 1))
            print()
        if i >= steps_per_epoch:
            break


    avg_loss = total_loss / (i + 1)
    print(f"Epoch [{epoch + 1}/{n_epochs}], Average Loss: {avg_loss}")
    print('\n\n\n')

i:  50
loss:  83.63920114554611

i:  100
loss:  71.14529264091266

i:  150
loss:  60.63262353353942

i:  200
loss:  53.54369906525114

i:  250
loss:  48.63179216726843

i:  300
loss:  44.803419246230014

Epoch [1/10], Average Loss: 41.931504634868105




i:  50
loss:  21.804274951710422

i:  100
loss:  21.556599494254236

i:  150
loss:  20.702452931183064

i:  200
loss:  20.096617784073104

i:  250
loss:  19.613032010446982

i:  300
loss:  19.109027571060334

Epoch [2/10], Average Loss: 18.73993757734326




i:  50
loss:  15.184262930178175

i:  100
loss:  15.249895766229912

i:  150
loss:  14.851211276275432

i:  200
loss:  14.60398600943646

i:  250
loss:  14.381918481621609

i:  300
loss:  14.111959333831686

Epoch [3/10], Average Loss: 13.908172107357691




i:  50
loss:  11.801751959557627

i:  100
loss:  11.944552742608703

i:  150
loss:  11.651383286280348

i:  200
loss:  11.465320686795819

i:  250
loss:  11.288965120733497

i:  300
loss:  11.095198493463256

Epoch [4/10], Aver

In [791]:
torch.save(model, 'siamese_network.pt')


# Evaluation  

### Evaluating your Siamese Network

In this section you will learn how to evaluate a Siamese network.

## Classify
To determine the accuracy of the model, we will utilize the test set that was configured earlier. While in training we used only positive examples, the test data, Q1_test, Q2_test and y_test, is setup as pairs of questions, some of which are duplicates some are not. 
This routine will run all the test question pairs through the model, compute the cosine simlarity of each pair, threshold it and compare the result to  y_test - the correct response from the data set. The results are accumulated to produce an accuracy.



### Exercise - classify

**Instructions**  
 - Loop through the incoming data in `batch_size` chunks
 - Use the data generator to load `q1`, `q2` a batch at a time.
 - copy a `batch_size` chunk of `y` into `y_test`
 - compute `v1`, `v2` using the model
 - for each element of the batch
        - compute the cos similarity of each pair of entries, `v1[j]`,`v2[j]`
        - determine if `d` > threshold
        - increment accuracy if that result matches the expected results (`y_test[j]`)
 - compute the final accuracy and return
 
Due to some limitations of this environment, running classify multiple times may result in the kernel failing. If that happens *Restart Kernal & clear output* and then run from the top. During development, consider using a smaller set of data to reduce the number of calls to model(). 

In [794]:
def classify(test_Q1, test_Q2, y, threshold, model, vocab, data_generator=data_generator, batch_size=64):
    """Function to test the accuracy of the model.

    Args:
        test_Q1 (numpy.ndarray): Array of Q1 questions.
        test_Q2 (numpy.ndarray): Array of Q2 questions.
        y (numpy.ndarray): Array of actual target.
        threshold (float): Desired threshold.
        model (trax.layers.combinators.Parallel): The Siamese model.
        vocab (collections.defaultdict): The vocabulary used.
        data_generator (function): Data generator function. Defaults to data_generator.
        batch_size (int, optional): Size of the batches. Defaults to 64.

    Returns:
        float: Accuracy of the model.
    """    
    
    
    accuracy = 0
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    for i in range(0, len(test_Q1), batch_size):
        # Call the data generator (built in Ex 01) with shuffle= None
        # use batch size chuncks of questions as Q1 & Q2 arguments of the data generator. e.g x[i:i + batch_size]
        # Hint: use `vocab['<PAD>']` for the `pad` argument of the data generator
        q1, q2 = next(data_generator(test_Q1[i:i+batch_size],test_Q2[i:i+batch_size],batch_size,vocab['<PAD>']))

        q1, q2 = torch.from_numpy(q1).long().to(device), torch.from_numpy(q2).long().to(device)
        # use batch size chuncks of actual output targets (same syntax as example above)
        y_test = y[i:i+batch_size]
        y_test = torch.from_numpy(y_test).long().to(device)


        # setting up initial hidden and cell state
        num_layers  = 3
        batch_size  = batch_size
        hidden_size = 128
        # Optionally, create initial hidden and cell states
        hidden_state = torch.zeros(num_layers, batch_size, hidden_size).to(device)
        memmory_cell = torch.zeros(num_layers, batch_size, hidden_size).to(device)
        # hidden_state.shape
    
        
        # Call the model    
        v1, _, _ = model(q1, hidden_state, memmory_cell)     # OR batch_of_output_vectors_1, hidden_state, memmory_cell
        v2, _, _ = model(q2, hidden_state, memmory_cell)     # OR batch_of_output_vectors_2, hidden_state, memmory_cell

        for j in range(batch_size):
            # take dot product to compute cos similarity of each pair of entries, v1[j], v2[j]
            # don't forget to transpose the second argument
            d = torch.matmul(v1[j],v2[j])   # yahan .T lenay ki zroorat nahi hay
            # is d greater than the threshold?
            res = d > threshold
            # increment accurancy if y_test is equal `res`
            accuracy += (y_test[j]==res)
    # compute accuracy using accuracy and total length of test questions
    accuracy = accuracy / len(test_Q1)
    ### END CODE HERE ###
    
    return accuracy


In [806]:
# this takes around 1 minute
accuracy = classify(Q1_test,Q2_test, y_test, 0.66, model, vocab, batch_size = 512) 
print("Accuracy", accuracy)


Accuracy tensor(0.7294, device='cuda:0')


In [34]:
# # this takes around 1 minute
# accuracy = classify(Q1_test,Q2_test, y_test, 0.7, model, vocab, batch_size = 512) 
# print("Accuracy", accuracy)

Accuracy 0.74423826


##### their result
Accuracy 0.74423826

**Expected Result**  
Accuracy ~0.74

<a name='5'></a>
## 5 -Testing with your Own Questions

In this section you will test the model with your own questions. You will write a function `predict` which takes two questions as input and returns $1$ or $0$ depending on whether the question pair is a duplicate or not.   

But first, we build a reverse vocabulary that allows to map encoded questions back to words: 

Write a function `predict`that takes in two questions, the model, and the vocabulary and returns whether the questions are duplicates ($1$) or not duplicates ($0$) given a similarity threshold. 

<a name='ex-6'></a>
### Exercise 6 - predict


**Instructions:** 
- Tokenize your question using `nltk.word_tokenize` 
- Create Q1,Q2 by encoding your questions as a list of numbers using vocab
- pad Q1,Q2 with next(data_generator([Q1], [Q2],1,vocab['<PAD>']))
- use model() to create v1, v2
- compute the cosine similarity (dot product) of v1, v2
- compute res by comparing d to the threshold


In [809]:
# UNQ_C6 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: predict
def predict(question1, question2, threshold, model, vocab, data_generator=data_generator, verbose=False):
    """Function for predicting if two questions are duplicates.

    Args:
        question1 (str): First question.
        question2 (str): Second question.
        threshold (float): Desired threshold.
        model (trax.layers.combinators.Parallel): The Siamese model.
        vocab (collections.defaultdict): The vocabulary used.
        data_generator (function): Data generator function. Defaults to data_generator.
        verbose (bool, optional): If the results should be printed out. Defaults to False.

    Returns:
        bool: True if the questions are duplicates, False otherwise.
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # use `nltk` word tokenize function to tokenize
    q1 = nltk.word_tokenize(question1)  # tokenize
    q2 = nltk.word_tokenize(question2)  # tokenize
    Q1, Q2 = [], []
    for word in q1:  # encode q1
        # append the 'word' index in `vocab`
        Q1.append(vocab[word])
    for word in q2:  # encode q2
        # append the 'word' index in `vocab`
        Q2.append(vocab[word])
        
    # Call the data generator (built in Ex 01) using next()
    # pass [Q1] & [Q2] as Q1 & Q2 arguments of the data generator. Set batch size as 1
    # Hint: use `vocab['<PAD>']` for the `pad` argument of the data generator
    Q1, Q2 = next(data_generator([Q1],[Q2],1,pad=vocab['<PAD>']))
    Q1, Q2 = torch.from_numpy(Q1).long().to(device), torch.from_numpy(Q2).long().to(device)
    
    # setting up initial hidden and cell state
    num_layers  = 3
    batch_size  = 1
    hidden_size = 128
    # Optionally, create initial hidden and cell states
    hidden_state = torch.zeros(num_layers, batch_size, hidden_size).to(device)
    memmory_cell = torch.zeros(num_layers, batch_size, hidden_size).to(device)
    # hidden_state.shape

    
    # Call the model
    v1, _, _ = model(Q1, hidden_state, memmory_cell)     # OR batch_of_output_vectors_1, hidden_state, memmory_cell
    v2, _, _ = model(Q2, hidden_state, memmory_cell)     # OR batch_of_output_vectors_2, hidden_state, memmory_cell


    # take dot product to compute cos similarity of each pair of entries, v1, v2
    # don't forget to transpose the second argument
    d = torch.matmul(v1,v2.T)
    # is d greater than the threshold?
    res = d > threshold
    
    ### END CODE HERE ###
    
    if(verbose):
        print("Q1  = ", Q1, "\nQ2  = ", Q2)
        print("d   = ", d)
        print("res = ", res)

    return res


In [810]:
# Feel free to try with your own questions
question1 = "When will I see you?"
question2 = "When can I see you again?"
# 1/True means it is duplicated, 0/False otherwise
predict(question1 , question2, 0.55, model, vocab, verbose = True)


Q1  =  tensor([[585,  76,   4,  46,  53,  21]], device='cuda:0') 
Q2  =  tensor([[ 585,   33,    4,   46,   53, 7280,   21]], device='cuda:0')
d   =  tensor([[0.8003]], device='cuda:0', grad_fn=<MmBackward0>)
res =  tensor([[True]], device='cuda:0')


tensor([[True]], device='cuda:0')

##### Expected Output
If input is:
```CPP
question1 = "When will I see you?"
question2 = "When can I see you again?"
```

Output is (d may vary a bit):
```CPP
Q1  =  [[585  76   4  46  53  21   1   1]] 
Q2  =  [[ 585   33    4   46   53 7280   21    1]]
d   =  0.8585515
res =  True
True
```

In [811]:
# Feel free to try with your own questions
question1 = "Do they enjoy eating the dessert?"
question2 = "Do they like hiking in the desert?"
# 1/True means it is duplicated, 0/False otherwise
predict(question1 , question2, 0.55, model, vocab, verbose=True) # 0.7 onhoon ny


Q1  =  tensor([[  443,  1145,  3159,  1169,    78, 29017,    21]], device='cuda:0') 
Q2  =  tensor([[  443,  1145,    60, 15302,    28,    78,  7431,    21]],
       device='cuda:0')
d   =  tensor([[0.4475]], device='cuda:0', grad_fn=<MmBackward0>)
res =  tensor([[False]], device='cuda:0')


tensor([[False]], device='cuda:0')

##### Expected output

If input is:
```CPP
question1 = "Do they enjoy eating the dessert?"
question2 = "Do they like hiking in the desert?"
```

Output  (d may vary a bit):

```CPP
Q1  =  [[  443  1145  3159  1169    78 29017    21     1]] 
Q2  =  [[  443  1145    60 15302    28    78  7431    21]]
d   =  0.53644466
res =  False
False
```

You can see that the Siamese network is capable of catching complicated structures. Concretely it can identify question duplicates although the questions do not have many words in common. 
 

In [813]:
# Feel free to try with your own questions
question1 = "Do you like getting up early in the morning?"
question2 = "Do you like sleeping late in the morning?"
# 1/True means it is duplicated, 0/False otherwise
predict(question1 , question2, 0.66, model, vocab, verbose=True) # 0.7 onhoon nay


Q1  =  tensor([[ 443,   53,   60,  128,  495,  986,   28,   78, 1432,   21]],
       device='cuda:0') 
Q2  =  tensor([[ 443,   53,   60, 1149,  960,   28,   78, 1432,   21]],
       device='cuda:0')
d   =  tensor([[0.5583]], device='cuda:0', grad_fn=<MmBackward0>)
res =  tensor([[False]], device='cuda:0')


tensor([[False]], device='cuda:0')

<a name='6'></a>
###  <span style="color:blue"> On Siamese Networks </span>

Siamese networks are important and useful. Many times there are several questions that are already asked in quora, or other platforms and you can use Siamese networks to avoid question duplicates. 

Congratulations, you have now built a powerful system that can recognize question duplicates. In the next course we will use transformers for machine translation, summarization, question answering, and chatbots. 
